In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interactive
from mpl_toolkits.mplot3d import Axes3D

# Spinning Basketball
## Lab 2

#### Instructions:
*Complete the notebook by reading all of the markdown cells and filling in the **blanks**. In the code cells, you will need to replace with the correct code everywhere you see a `___`. You may also need to add new code as appropriate. Please complete this lab during the lab slot and ask for help from the instructor or the TA as often as needed.*


## Introduction

Start by carefully watching the following video:

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("QtP_bh2lMXc")

The purpose of this lab is to analyze the motion of spinning ball dropped vertically from the top of Gordon Dam as shown in the video.  

The initial dropping of the basket ball shows that it deviates from falling straight down.  This is probably due to the wind.  Notwithstanding this potentially important effect, the analysis in this lab is ignore the effect of the wind.

From the two videos we have now seen regarding a basketball being dropped from The Gordon Dam, recall the two following statements:

- "Gordon dam is 126.5 m high."
- "When backspin was added ... the basketball went about 70 m horizontally."

## Theoretical Analysis

The dynamics of spinning ball under the influence of both a quadratic drag force and the Magnus force have been discussed in Lecture 6: Spinning Balls.  

Basketballs and physics go naturally together. In 2011, [The University Physics Competition](http://www.uphysicsc.com/2011contest.html) proposed the following problem:

> In the game of basketball, a player scores three points by successfully making a shot from beyond the three point line, which is 6.2 meters away from the basket in international games, such as the Olympics.  Suppose a player in an Olympic basketball game is at the three point line standing at a point making an angle 45 degrees to the principal axes of the court.   What initial ball velocities and spins will result in a successful shot from this point? 

A winning paper from that competition was

> Team 379: Rebecca Schutzengel, Patrick Varin, & Brendan Quinlivan
> Institution: F. W. Olin College of Engineering
> Team Advisor: Yevgeniya V. Zastavker
> [The 2011 Problem B Gold Medal Winning Paper: "The Physics of a Three Point Shot"](http://www.uphysicsc.com/2011-GM-B-379.PDF)

If you are interested, you may want to skim their award winning paper after this lab. In particular, note their discussion of the path through air when describing their model.  Their theory and assumptions are reproduced below:

>A spinning ball in the air is subject to three forces: gravity, drag and the Magnus force.
Therefore, the motion of the ball is governed by the following
equation:

\begin{align}
\frac{d^2 \vec{r}}{dt^2} =\vec{g} + \frac{1}{m_{ball}}\left( \frac{16}{3} \pi^2 r_{ball}^3  \rho \vec{\omega} \times \vec{v} - \frac{1}{2} \rho C_d  A |\vec{v}|^2 \hat{v} \right)
\end{align}

> where $\vec{r}$, $\vec{v}$, and $\vec{\omega}$ are the position, velocity, and angular
velocity of the basketball, $\vec{g}$ is the acceleration due to gravity,
$m_{ball}$, $r_{ball}$ and $A$ are the mass, radius, and cross-sectional
area of the basketball, $\rho$ is the density of air and $C_d$ is the
drag coefficient of the basketball in the air which we found
in literature to be 0.54. The first term in Equation 1
accounts for gravity, the second term accounts for the Magnus
force due to the ball’s spin  and the third term accounts for
the drag force. For simplicity, the basketball was assumed
to be spinning with a constant angular velocity throughout
its trajectory. It was also assumed that the spin was purely
backspin, meaning that the axis of rotation lies in the $xy$-plane
and is perpendicular to the ball’s trajectory

There is also the table of Model Parameters found at the end of their paper. 


|Basketball Property| Value |
|---|---|
| Circumference | 0.75 m|
| Mass | 0.600 kg |


While the paper is on different physical problem, their analysis serves as a useful starting point for our own simulation of a basketball projectile.

## Numerical model

Our goal will be to develop a numerical model to predict the trajectory of dropped basketball from an initial height $H$ and rotation rate $\omega$.  

### Initialize parameters

Starting by defining some variables for various physical parameters in the system. For each parameter need, type in a values from taken from either video or the paper. Be sure to include a comment describing the parameter and the units. Value for acceleration due to gravity and air density have already been added to show the expected format.

If you require additional physical parameters, please add them here as well.

In [ ]:
g = 9.81 # m/s^2, accleration due to gravity
ρ = 1.225 # kg/m^3, density of air
H = 1 # m, initial height
r = ___ # m, radius of a basketball
m = ___ # kg, mass of a basketball
Cd = ___ # drag coefficent (see the discussion of equation of motion above!)


### Numerical method

The code below is intended to solve equation of motion shown above using Euler's method. Fill in the missing parts to complete the implementation.  In order to do this, it may be helpful to work out the details on scrap paper first.   

In particular, you will need to calculate the values of $C_D$ and $C_M$ from other known physical properties according to the equation of motion from the basketball/physics paper (as given above). Recall that

$$ \frac{\vec{F_d}}{m} = - C_D |\vec{v}|\vec{v}$$

and

$$ \frac{\vec{F_M}}{m}= C_M \left(\vec{\omega} \times \vec{v}\right) $$

is how we have been modelling the drag force $F_d$ and the Magnus force $F_M$.

In [ ]:
def BasketballStepper(dt=0.01, 
                      x0=0, y0=0, z0=3,
                      vx0=10, vy0=0, vz0=0,
                      ωx=0, ωy=0, ωz=0,
                      tmax=4):
    """
    Solve for the motion of a spinning basketball with air drag using Euler's method
    
    x, y, z are the positions
    vx, vy, vz are the velocity components
    ωx, ωy, ωz are components of the angular velocity vector
    """
    
    # initialize the model
    t = 0
    x = x0
    y = y0
    z = z0
    vx = vx0
    vy = vy0
    vz = vz0
    
    while True:
        model = {'t': t, 
                 'x': x, 'y': y, 'z': z,
                 'vx': vx, 'vy': vy, 'vz': vz}
        yield model # return the model state back to the caller
    
        if t > tmax:
            break
        if z < 0: # stop if the ball hits the ground
            break
            
        # drag forces (per unit mass)
        v = np.sqrt(vx**2+vy**2+vz**2)
        # drag coefficient depends on speed
        CD = ___ # <<<<< From the equation of motion above
        Fdrag_x = -CD * v * vx
        Fdrag_y = -CD * v * vy
        Fdrag_z = -CD * v * vz
        
        # Magnus forces (per unit mass)
        CM = ___ # <<<<< From the equation of motion above
        Fmagnus_x = CM * (ωy*vz - ωz*vy)
        Fmagnus_y = CM * (ωz*vx - ωx*vz)
        Fmagnus_z = CM * (ωx*vy - ωy*vx)
        
        # net force (per unit mass)
        Fnet_x = Fdrag_x + Fmagnus_x
        Fnet_y = Fdrag_y + Fmagnus_y
        Fnet_z = Fdrag_z + Fmagnus_z - g
        
        # use the Euler algorithm to update the state of the model
        x = x + vx*dt
        y = y + vy*dt
        z = z + vz*dt
        vx = vx + Fnet_x * dt
        vy = vy + Fnet_y * dt
        vz = vz + Fnet_z * dt
        t = t + dt

In [ ]:
def BasketballPlot(ball):
    "Plot the trajector of a basketball"
    
    plt.plot(ball.x, ball.z)
    plt.xlabel('x (m)')
    plt.ylabel('z (m)')
    plt.axhline(0, color='k')

In [ ]:
def BasketballApp(ωy=0):
    "Model a dropped basketball with spin"
    
    # initial position
    x0 = 0
    y0 = 0
    z0 = H # m
    
    tmax = 10 # s
    dt = 0.01 # s
        
    # set up the model
    model = BasketballStepper(x0=x0, y0=y0, z0=z0,
                              vx0=0, vy0=0, vz0=0,
                              ωx=0, ωy=ωy, ωz=0,
                              tmax=tmax, dt=dt)
    
    # iterate the model
    ball = pd.DataFrame(model)

    # plot the results
    BasketballPlot(ball)

    return ball

In [ ]:
ball = BasketballApp()

## Analysis

### Estimating $\omega$
At the very end of the video, the presenter estimates the horizontal distance traveled by the basketball.  Let's call this observed distance $L_{obs}$ and define it to be the distance until the first bounce of the basketball off the water.  We may assume the water is at a height of $y = 0$ m. Fill in the estimated value of $L_{obs}$ below. 

In [ ]:
Lobs = ___ # m, horizontal distance travelled by basketball before first bounce

One parameter that we do not know is the rotation rate of the basketball.  Notice that the `BasketballApp()` function has an argument `ωy` that can be used to pass in a value for this rotation rate.

Using the `BasketballApp()` functions defined above, try and estimate the experimental value of $\omega$ that best matches the observed distance traveled.

1. Use `interactive` to try and determine the approximately correct value of $\omega_y$ (remember clockwise vs counterclockwise will change the sign of the angular velocity). 

2. Write a **new** program/application to make a plot of the the distance travelled to the first bounce verus angular velocity. Add a horizontal line (`plt.axhline()`) at the vertical position $L$ in your plot. Add a vertical line (`plt.axvline()`) at your best guess of $\omega_y$ for the experiment performed in the video.

The code below should be helpful in figuring out how far your ball travelled.

In [ ]:
def CalculateDistanceTravelled(ball):
    """
    Returns the horizontal distance travelled by the ball
    before it bounces (or the total distance if it never bounces)
    """
    
    # initial condition
    row1 = ball.iloc[0]
    xi = row1.x
    
    for row2 in ball.itertuples():
        if row1.vz < 0 and row2.vz > 0:
            # a "bounce" is determined when vz goes from <0 to >0.
            break
        row1 = row2
    xf = (row1.x + row2.x)/2
    
    L = xf-xi
    
    return L

ball = BasketballApp()
L = CalculateDistanceTravelled(ball)
plt.axvline(L, linestyle=':', color='r')

In [ ]:
def DistanceVsAngularVelocityApp():
    pass # 'pass' is used when a function needs to do nothing.
         # replace it with a new function!

DistanceVsAngularVelocityApp()

### Impact velocity

 Estimate the impact velocity. That is, what is the velocity when the basketball hits the water?

### Adding the bounce

Let's go back and improve the `BasketballStepper()` function so that instead of the basketball immediately coming to rest upon impact, it *bounces* off the water as shown in the video. 

The bounce of the basketball on the water is not perfectly elastic but there loss of energy and momentum (due to the water waves, acoustic waves, material properties of the ball, etc.).  As a parametrization, we could make the assumption that the velocity upon impact (which occurs if $z<0$) is modified as follows

\begin{align}
z &= - z \\
v_x &= \;\; \epsilon v_x \\
v_z &= - \epsilon v_z \\
\end{align}
for some unknown parameter $\epsilon$ with $0 \leq \epsilon \leq 1$.  The case $\epsilon = 1$ would be for a perfectly elastic collision on a rigid horizontal surface.

Go back and modify (only slightly) the `BasketballStepper()` function to include this effect. You'll will need to choose a value of $\epsilon$ (based on your the observations from the video).  Any plots or analysis you made above should not be affected by this change so double check that your changes did not break any other parts of this lab.

## Discussion
This lab is based on the assumption that the only forces on the ball were air drag, gravity, and the Magnus force.  Based on your analysis of the dynamics in the lab as compared to the observation of the trajectory in the video, do you think that our model reasonably captures the motion of the Magnus force?  If not, what other physics might we want to include in our model?

*your answer here...*